<a href="https://colab.research.google.com/github/leesolhahaha/study/blob/main/%EC%B4%88%EB%B3%B4_%EB%94%A5%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## 코드 오류

In [ ]:
import os
import glob
import librosa
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [ ]:
!pip install resampy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.3 MB/s eta 0:00:00


In [ ]:
## 특징 추출
import resampy

max_pad_len = 174

def extract_feature(file_name):
    print('file name :', file_name)
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0,0), (0, pad_width)), mode='constant')
        print(mfccs.shape)

    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        print(e);
        return None

#     return padded_mfccs
    return mfccs

In [ ]:
extract_feature('/content/drive/MyDrive/이브와/이브와ICT공모전/DATASET/audiofile/비명데이터셋/scream (1).wav')

# extract_feature('테스트할 음향 파일 위치')

file name : /content/drive/MyDrive/이브와/이브와ICT공모전/DATASET/audiofile/비명데이터셋/scream (1).wav
(40, 174)


array([[-5.8404602e+02, -5.7731555e+02, -5.9527911e+02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 6.7975349e+01,  7.5437355e+01,  6.1710022e+01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 7.7574735e+00,  7.9638152e+00,  1.0563673e+01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 1.7152834e-01,  1.4246166e-01,  8.7407136e-01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 1.5983510e+00,  2.9439249e+00,  1.4198196e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 1.8478974e+00,  3.5663905e+00,  5.1181822e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [ ]:
# Set the path to the full UrbanSound dataset
fulldatasetpath = '/content/drive/MyDrive/이브와/이브와ICT공모전/DATASET/audiofile'
metadata = pd.read_csv('/content/drive/MyDrive/엑셀파일/통합/위급통합.csv')
features = []

# Iterate through each sound file and extract the features
for index, row in metadata.iterrows():
    file_name = os.path.join(os.path.abspath(fulldatasetpath),str(row["fold"])+'/',str(row["slice_file_name"]))
    class_label = row['class_name']
    data = extract_feature(file_name)
    features.append([data, class_label])


# Convert into a Panda dataframe
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
file name : /content/drive/MyDrive/이브와/이브와ICT공모전/DATASET/audiofile/16.실외/16.실외_681176_label.wav
Error encountered while parsing file:  /content/drive/MyDrive/이브와/이브와ICT공모전/DATASET/audiofile/16.실외/16.실외_681176_label.wav
index can't contain negative values
file name : /content/drive/MyDrive/이브와/이브와ICT공모전/DATASET/audiofile/16.실외/16.실외_661989_label.wav
Error encountered while parsing file:  /content/drive/MyDrive/이브와/이브와ICT공모전/DATASET/audiofile/16.실외/16.실외_661989_label.wav
index can't contain negative values
file name : /content/drive/MyDrive/이브와/이브와ICT공모전/DATASET/audiofile/16.실외/16.실외_550228_label.wav
Error encountered while parsing file:  /content/drive/MyDrive/이브와/이브와ICT공모전/DATASET/audiofile/16.실외/16.실외_550228_label.wav
index can't contain negative values
file name : /content/drive/MyDrive/이브와/이브와ICT공모전/DATASET/audiofile/16.실외/16.실외_6

In [ ]:
# 피클로 데이터 저장
featuresdf.to_pickle("/content/drive/MyDrive/데이터셋/mfcc 이미지/featuresdf0.pkl")

In [ ]:
# 위급소음과 환경소음을 매핑하는 딕셔너리
mapping = {'비명': 0, '절도범죄': 0, '폭력범죄': 0, '실내': 1, '실외': 1, '갇힘': 0, '강제추행(성범죄)' : 0}  # 여기에 필요한 매핑 추가

# 기존 class_name 열 수정
featuresdf['class_label'] = featuresdf['class_label'].map(mapping)

# # 정수형으로 변환
# featuresdf['class_label'] = featuresdf['class_label'].astype(int)

# 결과 출력
print(featuresdf)

                                                feature  class_label
0     [[-614.9777, -242.51071, -144.6385, -111.03959...          NaN
1     [[-614.9777, -242.51071, -144.6385, -111.03959...          NaN
2     [[-590.09625, -289.16183, -139.16447, -55.3844...          NaN
3     [[-555.0338, -368.13788, -273.53647, -295.7059...          NaN
4     [[-555.0338, -368.13788, -273.53647, -295.7059...          NaN
...                                                 ...          ...
8831  [[-201.34692, -238.38164, -320.58893, -337.275...          NaN
8832  [[-478.11346, -427.76257, -258.38113, -205.947...          NaN
8833  [[-480.07242, -483.48856, -486.68402, -487.473...          NaN
8834  [[-481.01758, -489.94022, -400.25964, -270.762...          NaN
8835  [[-201.34692, -238.38164, -320.58893, -337.275...          NaN

[8836 rows x 2 columns]


In [ ]:
# NA 값 확인
print(featuresdf['class_label'].isna().sum())

# 무한대 값 확인
print(np.isinf(featuresdf['class_label']).sum())

8836
0


In [ ]:
from keras.utils import to_categorical

X = np.array(featuresdf.feature.tolist(), dtype=object)
y = np.array(featuresdf.class_label.tolist())

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [ ]:
print(len(x_train))
print(len(x_test))
print(x_train.shape)
print(x_test.shape, '\n')
print(y[:10])
print(yy[:10], '\n\n')
print(y_test[:10])

7068
1768
(7068,)
(1768,) 

['비명' '비명' '비명' '비명' '비명' '비명' '비명' '비명' '비명' '비명']
[[0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]] 


[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]


In [ ]:
n_columns = 174
n_row = 40
n_channels = 1
n_classes = 10

# input shape 조정
# cpu를 사용해서 수행한다
with tf.device('/cpu:0'):
    x_train = tf.reshape(x_train, [-1, n_row, n_columns, n_channels])
    x_test = tf.reshape(x_test, [-1, n_row, n_columns, n_channels])

ValueError: ignored